# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/smai/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/smai


In [4]:
!pip install numpy
!pip install tensorflow
!pip install re
!pip install nltk
!pip install keras
!pip install sklearn

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [5]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 32.0 MB/s 
     |████████████████████████████████| 1.2 MB 35.9 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 38.7 MB/s 
     |████████████████████████████████| 6.6 MB 39.2 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=370ec95b323d0e2891a3463ae572425d5f496644184189171de0a060b914c7c3
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# from zipfile import ZipFile
# file_name = "/content/q3_data.zip"

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()

In [7]:
train = pd.read_csv('q3_data/train.csv')
val = pd.read_csv('q3_data/val.csv')
test = pd.read_csv('q3_data/test.csv')

In [8]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [9]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [10]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [11]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [12]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [13]:
x_train[:5]

['say anni list polit group support third trimest abort demand abortion state representative texas republican',
 'declin coal start start natur ga took start begin presid georg w bush administr energy history job-accomplishments state delegate virginia democrat',
 'hillari clinton agre john mccain vote give georg bush benefit doubt iran foreign-policy president illinois democrat',
 'health care reform legisl like mandat free sex chang surgeri health-care no-job no-state none',
 'econom turnaround start end term economy jobs no-job florida democrat']

In [14]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [15]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [16]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [17]:
X.shape

(12836, 8000)

In [18]:
X[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [20]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [21]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [22]:
x_train_bow.shape

(10269, 8005)

In [23]:
x_train_bow[:5]

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   1,   1,   0],
       [  0,   0,   0, ..., 160, 163,   9],
       [  0,   0,   0, ...,   3,   5,  44],
       [  0,   0,   0, ...,  20,  19,   2]])

## Use of Glove Embedding


In [ ]:
!wget 'https://nlp.stanford.edu/data/glove.6B.zip'


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [25]:
!unzip "glove.6B.zip" -d "glove"

Archive:  glove.6B.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  


In [26]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [27]:
emmbed_dict['oov'] = np.zeros(200)

In [28]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [29]:
import tensorflow as tf

In [30]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    total_size=50
    corpus=np.array(corpus)
    corpus= tf.keras.preprocessing.sequence.pad_sequences(
    corpus,
    maxlen=total_size,
    padding='pre'
)
    return corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [31]:
# Reshaping the glove preprocesed data to match other dimenesion
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [32]:
print(x_train_glove.shape)

(10269, 50, 200)


In [33]:
x_train_glove=x_train_glove.reshape(x_train_glove.shape[0],x_train_glove.shape[1]*x_train_glove.shape[2])
x_val_glove=x_val_glove.reshape(x_val_glove.shape[0],x_val_glove.shape[1]*x_val_glove.shape[2])
x_test_glove=x_test_glove.reshape(x_test_glove.shape[0],x_test_glove.shape[1]*x_test_glove.shape[2])

In [34]:
# Printing the shape of the data
print(x_train_glove.shape)
print(x_val_glove.shape)
print(x_test_glove.shape)
print(x_train2.shape)
print(x_val2.shape)
print(x_test2.shape)


(10269, 10000)
(1284, 10000)
(1283, 10000)
(10269, 5)
(1284, 5)
(1283, 5)


In [35]:
# Reshaping the data to resolve errors and caching issue in ram
x_train_glove = np.hstack((x_train_glove, x_train2))
x_val_glove = np.hstack((x_val_glove, x_val2))
x_test_glove = np.hstack((x_test_glove, x_test2))

In [36]:
print(x_train_glove.shape)

(10269, 10005)


## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [37]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [38]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [39]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)
print(len(y_train_six_way))

10269


Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [40]:
from keras.models import Sequential
from sklearn.metrics import classification_report,accuracy_score, precision_score, recall_score, f1_score,confusion_matrix,multilabel_confusion_matrix
from keras.layers import LSTM,Dense, Dropout, Embedding, Bidirectional
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping


##### Model for bert embeddings(Six way classification)

> Indented block



In [ ]:
#BERT
## write your code here
# Initialize hyperparameters
# Create model
model_bert = Sequential()
model_bert.add(LSTM(80, return_sequences = True, input_shape = (x_train_bert.shape[1], 1)))
model_bert.add(Dropout(0.2))
model_bert.add(LSTM(50))
model_bert.add(Dropout(0.3))
model_bert.add(Dense(6, activation='softmax'))
model_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=2,restore_best_weights=True)
# train
history =model_bert.fit(x_train_bert, y_train_six_way, validation_data=(x_val_bert, y_val_six_way), batch_size = 128, epochs = 10,callbacks=[earlyStop])

Epoch 1/12
81/81 [==============================] - 15s 137ms/step - loss: 1.7596 - accuracy: 0.2080 - val_loss: 1.7518 - val_accuracy: 0.2048
Epoch 2/12
81/81 [==============================] - 10s 126ms/step - loss: 1.7394 - accuracy: 0.2209 - val_loss: 1.7206 - val_accuracy: 0.2578
Epoch 3/12
81/81 [==============================] - 10s 126ms/step - loss: 1.7118 - accuracy: 0.2444 - val_loss: 1.6733 - val_accuracy: 0.2866
Epoch 4/12
81/81 [==============================] - 10s 126ms/step - loss: 1.6615 - accuracy: 0.2875 - val_loss: 1.6214 - val_accuracy: 0.3326
Epoch 5/12
81/81 [==============================] - 10s 126ms/step - loss: 1.5755 - accuracy: 0.3391 - val_loss: 1.4747 - val_accuracy: 0.3972
Epoch 6/12
81/81 [==============================] - 10s 126ms/step - loss: 1.4566 - accuracy: 0.3991 - val_loss: 1.3684 - val_accuracy: 0.4283
Epoch 7/12
81/81 [==============================] - 10s 126ms/step - loss: 1.3839 - accuracy: 0.4267 - val_loss: 1.3354 - val_accuracy: 0.4431

In [ ]:
# test
print("Result on test data")
print("loss on test data, test data accuracy :", model_bert.evaluate(x_test_bert, y_test_six_way))

Result on test data
41/41 [==============================] - 2s 60ms/step - loss: 1.3015 - accuracy: 0.4435
loss on test data, test data accuracy : [1.3014678955078125, 0.4434918165206909]


In [ ]:
# report accuracy, f1-score and confusion matrix
y_hats =model_bert.predict(x_test_bert)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))

Confusion Matrix
 [[129   2  19  53  31  16]
 [ 30  33  23  49  73   3]
 [ 42   0  73  65  28   6]
 [ 33   1  18 150  61   4]
 [ 22   0  22  69 132   4]
 [ 19   0   5  12   4  52]]

Accuracy on Bert embedded inputs : 0.4435


 f1 score on Bert embedded inputs : 0.4407


Classification Report of Bert embedded inputs 

              precision    recall  f1-score   support

       False       0.47      0.52      0.49       250
        True       0.92      0.16      0.27       211
 barely-true       0.46      0.34      0.39       214
   half-true       0.38      0.56      0.45       267
 mostly-true       0.40      0.53      0.46       249
  pants-fire       0.61      0.57      0.59        92

    accuracy                           0.44      1283
   macro avg       0.54      0.45      0.44      1283
weighted avg       0.52      0.44      0.43      1283



##### Model for  Bag of words (Six Way classification)

In [ ]:
#BOW
## write your code here
# Initialize hyperparameters
# Create model
model_bow = Sequential()
model_bow.add(LSTM(80, return_sequences = True, input_shape = (x_train_bow.shape[1], 1)))
model_bow.add(Dropout(0.2))
model_bow.add(LSTM(50))
model_bow.add(Dropout(0.3))
model_bow.add(Dense(6, activation='softmax'))
model_bow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=2,restore_best_weights=True)
history =model_bow.fit(x_train_bow, y_train_six_way, validation_data=(x_val_bow, y_val_six_way), batch_size = 64, epochs = 10,callbacks=[earlyStop])
# test

# report accuracy, f1-score and confusion matrix


Epoch 1/10
161/161 [==============================] - 256s 2s/step - loss: 1.7585 - accuracy: 0.2098 - val_loss: 1.7354 - val_accuracy: 0.2453
Epoch 2/10
161/161 [==============================] - 251s 2s/step - loss: 1.7184 - accuracy: 0.2388 - val_loss: 1.6688 - val_accuracy: 0.2983
Epoch 3/10
161/161 [==============================] - 247s 2s/step - loss: 1.6510 - accuracy: 0.2913 - val_loss: 1.5669 - val_accuracy: 0.3832
Epoch 4/10
161/161 [==============================] - 242s 2s/step - loss: 1.5374 - accuracy: 0.3675 - val_loss: 1.4502 - val_accuracy: 0.4206
Epoch 5/10
161/161 [==============================] - 241s 1s/step - loss: 1.4402 - accuracy: 0.4051 - val_loss: 1.3707 - val_accuracy: 0.4361
Epoch 6/10
161/161 [==============================] - 241s 1s/step - loss: 1.3734 - accuracy: 0.4283 - val_loss: 1.3275 - val_accuracy: 0.4556
Epoch 7/10
161/161 [==============================] - 241s 1s/step - loss: 1.3391 - accuracy: 0.4374 - val_loss: 1.2989 - val_accuracy: 0.4478

In [ ]:
print("Result on test data")
print("loss on test data, test data accuracy :", model_bow.evaluate(x_test_bow, y_test_six_way))
y_hats =model_bow.predict(x_test_bow)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bag of words embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bag of words embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bag of words embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))

Result on test data
41/41 [==============================] - 72s 2s/step - loss: 1.7273 - accuracy: 0.2011
loss on test data, test data accuracy : [1.7272846698760986, 0.20109118521213531]
Confusion Matrix
 [[ 15   0   0  59 167   9]
 [ 10   0   0  31 168   2]
 [ 14   0   0  49 149   2]
 [ 16   0   0  56 194   1]
 [ 11   0   0  62 176   0]
 [ 11   0   0  16  54  11]]

Accuracy on Bert embedded inputs : 0.2011


 f1 score on Bert embedded inputs : 0.1319


Classification Report of Bert embedded inputs 

              precision    recall  f1-score   support

       False       0.19      0.06      0.09       250
        True       0.00      0.00      0.00       211
 barely-true       0.00      0.00      0.00       214
   half-true       0.21      0.21      0.21       267
 mostly-true       0.19      0.71      0.30       249
  pants-fire       0.44      0.12      0.19        92

    accuracy                           0.20      1283
   macro avg       0.17      0.18      0.13      1283
weig

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Model for Glove Embeddings ( Six way classification)

In [42]:
#glove
## write your code here
# Initialize hyperparameters
# Create model
model_glove = Sequential()
model_glove.add(LSTM(80, return_sequences = True, input_shape = (x_train_glove.shape[1], 1)))
model_glove.add(Dropout(0.2))
model_glove.add(LSTM(50))
model_glove.add(Dropout(0.3))
model_glove.add(Dense(6, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=2,restore_best_weights=True)
history =model_glove.fit(x_train_glove, y_train_six_way, validation_data=(x_val_glove, y_val_six_way), batch_size = 128, epochs = 10,callbacks=[earlyStop])

Epoch 1/10
81/81 [==============================] - 179s 2s/step - loss: 1.7641 - accuracy: 0.2038 - val_loss: 1.7531 - val_accuracy: 0.1939
Epoch 2/10
81/81 [==============================] - 173s 2s/step - loss: 1.7391 - accuracy: 0.2225 - val_loss: 1.7135 - val_accuracy: 0.2461
Epoch 3/10
81/81 [==============================] - 172s 2s/step - loss: 1.6975 - accuracy: 0.2678 - val_loss: 1.6496 - val_accuracy: 0.3154
Epoch 4/10
81/81 [==============================] - 172s 2s/step - loss: 1.6196 - accuracy: 0.3194 - val_loss: 1.5206 - val_accuracy: 0.4229
Epoch 5/10
81/81 [==============================] - 172s 2s/step - loss: 1.4765 - accuracy: 0.3978 - val_loss: 1.3848 - val_accuracy: 0.4377
Epoch 6/10
81/81 [==============================] - 172s 2s/step - loss: 1.3772 - accuracy: 0.4301 - val_loss: 1.3279 - val_accuracy: 0.4470
Epoch 7/10
81/81 [==============================] - 171s 2s/step - loss: 1.3480 - accuracy: 0.4290 - val_loss: 1.3065 - val_accuracy: 0.4595
Epoch 8/10
81

In [43]:
# test
y_hats =model_glove.predict(x_test_glove)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
# report accuracy, f1-score and confusion matrix
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on glove embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on glove embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of glove embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))

Confusion Matrix
 [[109   2  20  40  59  20]
 [ 29  33  16  30 100   3]
 [ 33   0  67  36  63  15]
 [ 33   1  14 102 113   4]
 [ 20   0   6  41 177   5]
 [ 14   0   3  12   4  59]]

Accuracy on glove embedded inputs : 0.4263


 f1 score on glove embedded inputs : 0.4255


Classification Report of glove embedded inputs 

              precision    recall  f1-score   support

       False       0.46      0.44      0.45       250
        True       0.92      0.16      0.27       211
 barely-true       0.53      0.31      0.39       214
   half-true       0.39      0.38      0.39       267
 mostly-true       0.34      0.71      0.46       249
  pants-fire       0.56      0.64      0.60        92

    accuracy                           0.43      1283
   macro avg       0.53      0.44      0.43      1283
weighted avg       0.52      0.43      0.41      1283



# Binary Classification

## Preprocessing

In [44]:
num_classes = 2

In [45]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [46]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


##### Model for bert Embedding ( Binary )

In [58]:
## write your code here
# Initialize hyperparameters
# Create model
## write your code here
# Initialize hyperparameters
# Create model
model_bert_bin = Sequential()
model_bert_bin.add(LSTM(80, return_sequences = True, input_shape = (x_train_bert.shape[1], 1)))
model_bert_bin.add(Dropout(0.2))
model_bert_bin.add(LSTM(50))
model_bert_bin.add(Dropout(0.3))
model_bert_bin.add(Dense(2, activation='softmax'))
model_bert_bin.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=2,restore_best_weights=True)
history =model_bert_bin.fit(x_train_bert, y_train_binary, validation_data=(x_val_bert, y_val_binary), batch_size = 128,epochs = 10,callbacks=[earlyStop])

Epoch 1/10
81/81 [==============================] - 16s 151ms/step - loss: 0.6843 - accuracy: 0.5636 - val_loss: 0.6805 - val_accuracy: 0.5724
Epoch 2/10
81/81 [==============================] - 10s 129ms/step - loss: 0.6654 - accuracy: 0.5991 - val_loss: 0.6582 - val_accuracy: 0.6254
Epoch 3/10
81/81 [==============================] - 10s 122ms/step - loss: 0.6386 - accuracy: 0.6387 - val_loss: 0.6280 - val_accuracy: 0.6402
Epoch 4/10
81/81 [==============================] - 10s 119ms/step - loss: 0.6067 - accuracy: 0.6765 - val_loss: 0.5821 - val_accuracy: 0.6815
Epoch 5/10
81/81 [==============================] - 10s 119ms/step - loss: 0.5633 - accuracy: 0.7077 - val_loss: 0.5692 - val_accuracy: 0.6838
Epoch 6/10
81/81 [==============================] - 10s 119ms/step - loss: 0.5474 - accuracy: 0.7101 - val_loss: 0.5404 - val_accuracy: 0.6885
Epoch 7/10
81/81 [==============================] - 10s 119ms/step - loss: 0.5358 - accuracy: 0.7164 - val_loss: 0.5290 - val_accuracy: 0.6955

In [59]:
# test
print("Result on test data")
print("loss on test data, test data accuracy :", model_bert_bin.evaluate(x_test_bert, y_test_binary))
y_hats =model_bert_bin.predict(x_test_bert)
actual_y=np.argmax(y_test_binary,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True']))
# report accuracy, f1-score and confusion matrix
# train
# test
# report accuracy, f1-score and confusion matrix

Result on test data
41/41 [==============================] - 2s 59ms/step - loss: 0.5158 - accuracy: 0.7389
loss on test data, test data accuracy : [0.5157830715179443, 0.7388932108879089]
Confusion Matrix
 [[344 212]
 [123 604]]

Accuracy on Bert embedded inputs : 0.7389


 f1 score on Bert embedded inputs : 0.7277


Classification Report of Bert embedded inputs 

              precision    recall  f1-score   support

       False       0.74      0.62      0.67       556
        True       0.74      0.83      0.78       727

    accuracy                           0.74      1283
   macro avg       0.74      0.72      0.73      1283
weighted avg       0.74      0.74      0.74      1283



##### Model for Bag of Words ( Binary )

In [52]:
## write your code here
# Initialize hyperparameters
# Create model
## write your code here
# Initialize hyperparameters
# Create model
model_bow_bin = Sequential()
model_bow_bin.add(LSTM(80, return_sequences = True, input_shape = (x_train_bow.shape[1], 1)))
model_bow_bin.add(Dropout(0.2))
model_bow_bin.add(LSTM(50))
model_bow_bin.add(Dropout(0.3))
model_bow_bin.add(Dense(2, activation='softmax'))
model_bow_bin.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=2,restore_best_weights=True)
history =model_bow_bin.fit(x_train_bow, y_train_binary, validation_data=(x_val_bow, y_val_binary), batch_size = 64, epochs = 12,callbacks=[earlyStop])


Epoch 1/12
161/161 [==============================] - 271s 2s/step - loss: 0.6751 - accuracy: 0.5799 - val_loss: 0.6600 - val_accuracy: 0.5950
Epoch 2/12
161/161 [==============================] - 267s 2s/step - loss: 0.6314 - accuracy: 0.6458 - val_loss: 0.6038 - val_accuracy: 0.6464
Epoch 3/12
161/161 [==============================] - 266s 2s/step - loss: 0.5882 - accuracy: 0.6950 - val_loss: 0.5763 - val_accuracy: 0.6963
Epoch 4/12
161/161 [==============================] - 265s 2s/step - loss: 0.5682 - accuracy: 0.7072 - val_loss: 0.5539 - val_accuracy: 0.6994
Epoch 5/12
161/161 [==============================] - 261s 2s/step - loss: 0.5712 - accuracy: 0.6992 - val_loss: 0.6980 - val_accuracy: 0.5927
Epoch 6/12
161/161 [==============================] - 256s 2s/step - loss: 0.6427 - accuracy: 0.6358 - val_loss: 0.6624 - val_accuracy: 0.5927
Epoch 6: early stopping


In [55]:
# test
print("Result on test data")
print("loss on test data, test data accuracy :", model_bow_bin.evaluate(x_test_bow, y_test_binary))
y_hats =model_bow_bin.predict(x_test_bow)
actual_y=np.argmax(y_test_binary,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True']))
# report accuracy, f1-score and confusion matrix
# train
# test
# report accuracy, f1-score and confusion matrix

Result on test data
41/41 [==============================] - 27s 661ms/step - loss: 0.5575 - accuracy: 0.7085
loss on test data, test data accuracy : [0.5575038194656372, 0.7084957361221313]
Confusion Matrix
 [[324 232]
 [142 585]]

Accuracy on Bert embedded inputs : 0.7085


 f1 score on Bert embedded inputs : 0.6959


Classification Report of Bert embedded inputs 

              precision    recall  f1-score   support

       False       0.70      0.58      0.63       556
        True       0.72      0.80      0.76       727

    accuracy                           0.71      1283
   macro avg       0.71      0.69      0.70      1283
weighted avg       0.71      0.71      0.70      1283



##### Model for Glove ( Binary )

In [60]:
## write your code here
# Initialize hyperparameters
# Create model
model_glove_bin = Sequential()
model_glove_bin.add(LSTM(80, return_sequences = True, input_shape = (x_train_glove.shape[1], 1)))
model_glove_bin.add(Dropout(0.2))
model_glove_bin.add(LSTM(50))
model_glove_bin.add(Dropout(0.3))
model_glove_bin.add(Dense(2, activation='softmax'))
model_glove_bin.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_glove_bin.fit(x_train_glove, y_train_binary, validation_data=(x_val_glove, y_val_binary), batch_size = 128, epochs = 10)

Epoch 1/10
81/81 [==============================] - 176s 2s/step - loss: 0.6829 - accuracy: 0.5699 - val_loss: 0.6830 - val_accuracy: 0.5537
Epoch 2/10
81/81 [==============================] - 172s 2s/step - loss: 0.6602 - accuracy: 0.6073 - val_loss: 0.6526 - val_accuracy: 0.6285
Epoch 3/10
81/81 [==============================] - 171s 2s/step - loss: 0.6308 - accuracy: 0.6455 - val_loss: 0.6126 - val_accuracy: 0.6495
Epoch 4/10
81/81 [==============================] - 171s 2s/step - loss: 0.5932 - accuracy: 0.6878 - val_loss: 0.5915 - val_accuracy: 0.6791
Epoch 5/10
81/81 [==============================] - 171s 2s/step - loss: 0.5616 - accuracy: 0.7095 - val_loss: 0.5482 - val_accuracy: 0.6900
Epoch 6/10
81/81 [==============================] - 171s 2s/step - loss: 0.5447 - accuracy: 0.7101 - val_loss: 0.5357 - val_accuracy: 0.6916
Epoch 7/10
81/81 [==============================] - 171s 2s/step - loss: 0.5327 - accuracy: 0.7167 - val_loss: 0.5265 - val_accuracy: 0.6924
Epoch 8/10
81

In [61]:
# test

y_hats =model_glove_bin.predict(x_test_glove)
actual_y=np.argmax(y_test_binary,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
# report accuracy, f1-score and confusion matrix
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on glove embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on glove embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of glove embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True']))

Confusion Matrix
 [[315 241]
 [ 96 631]]

Accuracy on glove embedded inputs : 0.7373


 f1 score on glove embedded inputs : 0.7204


Classification Report of glove embedded inputs 

              precision    recall  f1-score   support

       False       0.77      0.57      0.65       556
        True       0.72      0.87      0.79       727

    accuracy                           0.74      1283
   macro avg       0.75      0.72      0.72      1283
weighted avg       0.74      0.74      0.73      1283

